In [ ]:
import pandas as pd
data=pd.read_csv('/content/drive/MyDrive/bbc_news_mixed (1).csv')

In [ ]:
data.head()

,text,label
0,Cairn shares slump on oil setback\n\nShares in...,business
1,Egypt to sell off state-owned bank\n\nThe Egyp...,business
2,Cairn shares up on new oil find\n\nShares in C...,business
3,Low-cost airlines hit Eurotunnel\n\nChannel Tu...,business
4,"Parmalat to return to stockmarket\n\nParmalat,...",business


In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import LabelBinarizer
data.label.value_counts()

sport            511
business         510
politics         417
tech             401
entertainment    386
Name: label, dtype: int64

In [ ]:
label2= pd.get_dummies(data["label"])

In [ ]:
label2.head()

,business,entertainment,politics,sport,tech
0,1,0,0,0,0
1,1,0,0,0,0
2,1,0,0,0,0
3,1,0,0,0,0
4,1,0,0,0,0


In [ ]:
y = LabelBinarizer().fit_transform(label2)

In [ ]:
y[:5]

array([[1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0],
       [1, 0, 0, 0, 0]])

In [ ]:
z= pd.DataFrame(y)
z.value_counts()

0  1  2  3  4
0  0  0  1  0    511
1  0  0  0  0    510
0  0  1  0  0    417
      0  0  1    401
   1  0  0  0    386
dtype: int64

In [ ]:
data.head()

,text,label
0,Cairn shares slump on oil setback\n\nShares in...,business
1,Egypt to sell off state-owned bank\n\nThe Egyp...,business
2,Cairn shares up on new oil find\n\nShares in C...,business
3,Low-cost airlines hit Eurotunnel\n\nChannel Tu...,business
4,"Parmalat to return to stockmarket\n\nParmalat,...",business


In [ ]:
from gensim.utils import simple_preprocess
preprocessed_bbc = data.text.apply(lambda x: simple_preprocess(x))
preprocessed_bbc.head()

0    [cairn, shares, slump, on, oil, setback, share...
1    [egypt, to, sell, off, state, owned, bank, the...
2    [cairn, shares, up, on, new, oil, find, shares...
3    [low, cost, airlines, hit, eurotunnel, channel...
4    [parmalat, to, return, to, stockmarket, parmal...
Name: text, dtype: object

In [ ]:
# import word2vec
from gensim.models import Word2Vec

# train a word2vec model from the given data set
w2v_model = Word2Vec(preprocessed_bbc, size=300, min_count=2, sg=1)

In [ ]:
print('vocabulary size:', len(w2v_model.wv.vocab))

vocabulary size: 18588


In [ ]:
w2v_model.wv.most_similar('oil')

[('gas', 0.8566049337387085),
 ('telecoms', 0.8119033575057983),
 ('costs', 0.8108860850334167),
 ('giant', 0.8099585175514221),
 ('unit', 0.7962621450424194),
 ('fuel', 0.7939218282699585),
 ('energy', 0.7836349606513977),
 ('industrial', 0.7809122800827026),
 ('steel', 0.7744253873825073),
 ('exports', 0.7730951309204102)]

In [ ]:
w2v_model.save('/content/drive/MyDrive/bbc_w2v_model.h5')

In [ ]:
def get_embedding_w2v(doc_tokens, pre_trained):
    embeddings = []
    if pre_trained:
        model = w2vec
    else:
        model = w2v_model
    for tok in doc_tokens:
        if tok in model.wv.vocab:
            embeddings.append(model.wv.word_vec(tok))
    return np.mean(embeddings, axis=0)

In [ ]:
import numpy as np
X_w2v_model = preprocessed_bbc.apply(lambda x: get_embedding_w2v(x, pre_trained=0))
X_w2v_model = pd.DataFrame(X_w2v_model.tolist())
print('X shape:', X_w2v_model.shape)

X shape: (2225, 300)


In [ ]:
from sklearn.model_selection import train_test_split
X_train_wm, X_test_wm, y_train_wm, y_test_wm = train_test_split(X_w2v_model, y)

In [ ]:
X_train_wm.shape,X_test_wm.shape

((1668, 300), (557, 300))

In [ ]:
y_train_wm.shape, y_test_wm.shape

((1668, 5), (557, 5))

In [ ]:
X_train_wm=np.array(X_train_wm).reshape(1668, 300,1)
X_train_wm.shape

(1668, 300, 1)

In [ ]:
X_test_wm=np.array(X_test_wm).reshape(557, 300,1)
X_test_wm.shape

(557, 300, 1)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Dense

model1=Sequential()
model1.add(LSTM(100,input_shape=(300,1)))
model1.add(Dense(8,activation="relu"))
model1.add(Dense(5,activation="softmax"))

model1.compile(loss="categorical_crossentropy",optimizer="adam",metrics=["accuracy"])

In [ ]:
model1.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 100)               40800     
_________________________________________________________________
dense_21 (Dense)             (None, 8)                 808       
_________________________________________________________________
dense_22 (Dense)             (None, 5)                 45        
Total params: 41,653
Trainable params: 41,653
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model1.fit(X_train_wm, y_train_wm,validation_data=(X_test_wm, y_test_wm), epochs=100)

Epoch 1/100
53/53 [==============================] - 2s 25ms/step - loss: 1.6096 - accuracy: 0.2029 - val_loss: 1.6071 - val_accuracy: 0.2154
Epoch 2/100
53/53 [==============================] - 1s 16ms/step - loss: 1.6076 - accuracy: 0.2217 - val_loss: 1.6061 - val_accuracy: 0.2154
Epoch 3/100
53/53 [==============================] - 1s 16ms/step - loss: 1.6062 - accuracy: 0.2239 - val_loss: 1.6049 - val_accuracy: 0.2154
Epoch 4/100
53/53 [==============================] - 1s 16ms/step - loss: 1.6066 - accuracy: 0.2295 - val_loss: 1.6038 - val_accuracy: 0.2154
Epoch 5/100
53/53 [==============================] - 1s 16ms/step - loss: 1.6056 - accuracy: 0.2147 - val_loss: 1.6032 - val_accuracy: 0.2154
Epoch 6/100
53/53 [==============================] - 1s 16ms/step - loss: 1.6008 - accuracy: 0.2396 - val_loss: 1.5978 - val_accuracy: 0.2154
Epoch 7/100
53/53 [==============================] - 1s 16ms/step - loss: 1.5989 - accuracy: 0.2406 - val_loss: 1.5985 - val_accuracy: 0.2172
Epoch 

In [ ]:
model1.save('/content/drive/MyDrive/bbc_news_model.h5')

In [ ]:
model1.evaluate(X_test_wm,y_test_wm)

18/18 [==============================] - 0s 8ms/step - loss: 0.4780 - accuracy: 0.8348


[0.47802647948265076, 0.8348294496536255]

In [ ]:
#classes=["business",	"entertainment",	"politics",	"sport",	"tech"]

In [ ]:
def prediction(doc):
  classes=["business",	"entertainment",	"politics",	"sport",	"tech"]
  doc= simple_preprocess(doc)
  doc= get_embedding_w2v(doc, pre_trained=0)
  doc1= doc.reshape(1,300,1)
  p= model1.predict(doc1)
  return classes[np.argmax(p)]

In [ ]:
doc1= "Pankaj Tripathi, currently seen on Criminal Justice: Behind Closed Doors, opens up about dealing with fame"
print(f"The article belongs to {prediction(doc1)} category",)

The article belongs to tech category


In [ ]:
doc2= "OnePlus 9 Alleged Live Images Tip Flat Hole-Punch Display, Reverse Wireless Charging Support"
print(f"The article belongs to {prediction(doc2)} category",)

The article belongs to tech category


In [ ]:
doc3= "Tesla public company duties are a much bigger factor, but going private is impossible now (sigh),” Musk said in response to a tweet saying he should optimize his time in areas such as innovation"
print(f"The article belongs to {prediction(doc3)} category",)

The article belongs to politics category


In [ ]:
doc4= "PSG sack Tuchel, Pochettino set to become new manager - reports."
print(f"The article belongs to {prediction(doc4)} category",)

The article belongs to sport category


In [ ]:
doc5= "In a press conference on Tuesday, Kejriwal said the development of Uttar Pradesh has been held back by 'corrupt' leaders in the state"
print(f"The article belongs to {prediction(doc5)} category",)

The article belongs to business category


In [ ]:
doc6= "RIL plans to rebrand the IMG Reliance as its completely owned subsidiary post-acquisition of 50 per cent shares held by the wholly-owned subsidiary of IMG, the MG Singapore Pte. Ltd."
print(f"The article belongs to {prediction(doc6)} category",)

The article belongs to business category


 **Loading the saved model**

In [ ]:
from tensorflow.keras.models import load_model
bbc = load_model('/content/drive/MyDrive/bbc_news_model.h5')

In [ ]:
#embed = load_model('/content/drive/MyDrive/bbc_w2v_model.h5')
import gensim
embed = gensim.models.Word2Vec.load('/content/drive/MyDrive/bbc_w2v_model.h5')

In [ ]:
def get_embedding_w2v_bbc(doc_tokens):
    embeddings = []
    for tok in doc_tokens:
        if tok in embed.wv.vocab:
            embeddings.append(embed.wv.word_vec(tok))
    return np.mean(embeddings, axis=0)

In [ ]:
def predictionbbc(doc):
  classes=["business",	"entertainment",	"politics",	"sport",	"tech"]
  doc= simple_preprocess(doc)
  doc= get_embedding_w2v_bbc(doc)
  doc1= doc.reshape(1,300,1)
  p= bbc.predict(doc1)
  return classes[np.argmax(p)]

In [ ]:
test_doc= "PSG sack Tuchel, Pochettino set to become new manager - reports."
print(f"The article belongs to {predictionbbc(test_doc)} category",)

The article belongs to sport category
